In [1]:
from ultralytics import YOLO
import cv2
import cvzone
import math
import cv2
import numpy as np

In [2]:

model=YOLO("../Running Yolo/yolov8l.pt")
# DOWNLOAD YOLO MODEL ACCORDING TO YOUR HARDWARE CAPABILITIES . this is a large model running on GPU, you can run on it CPU but nano model is best suited 
#for CPU
coco_classes = [
    "Person", "Bicycle", "Car", "Motorcycle", "Airplane", "Bus", "Train", "Truck", "Boat", "Traffic light",
    "Fire hydrant", "Stop sign", "Parking meter", "Bench", "Bird", "Cat", "Dog", "Horse", "Sheep", "Cow",
    "Elephant", "Bear", "Zebra", "Giraffe", "Backpack", "Umbrella", "Handbag", "Tie", "Suitcase", "Frisbee",
    "Skis", "Snowboard", "Sports ball", "Kite", "Baseball bat", "Baseball glove", "Skateboard", "Surfboard",
    "Tennis racket", "Bottle", "Wine glass", "Cup", "Fork", "Knife", "Spoon", "Bowl", "Banana", "Apple",
    "Sandwich", "Orange", "Broccoli", "Carrot", "Hot dog", "Pizza", "Donut", "Cake", "Chair", "Couch",
    "Potted plant", "Bed", "Dining table", "Toilet", "TV", "Laptop", "Mouse", "Remote", "Keyboard",
    "Cell phone", "Microwave", "Oven", "Toaster", "Sink", "Refrigerator", "Book", "Clock", "Vase", "Scissors",
    "Teddy bear", "Hair drier", "Toothbrush"
]

In [3]:
import torch

# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    # Get the current device (GPU)
    device = torch.device("cuda")
    print("GPU is available")
else:
    # Use CPU
    device = torch.device("cpu")
    print("GPU is not available, using CPU")

print("Current device:", device)

GPU is available
Current device: cuda


In [4]:
import os

# Get the current working directory
current_directory = os.getcwd()

# Print the current working directory
print("Current Directory:", current_directory)

Current Directory: D:\Conda_Pro_\Objectdetection\People Counter


## For putting a tracker 
We will be using a tracker from github repository: https://github.com/abewley/sort/blob/master/sort.py

In [5]:
from sort import *

## The limits up-down have been made according to the stock footage. 
I wanted to count to people going up and down so I made limits up & down giving the coordinates of the lines which act as the counter of people.
You may need to change this according to the footage

In [6]:
# Tracker
tracker = Sort(max_age=1, min_hits=3, iou_threshold=0.3)
limitsUp =[103,161,296,161]
limitsDown =[527,489,735,489]

In [7]:
mask = cv2.imread("mask.png",cv2.IMREAD_GRAYSCALE)
mask = mask.astype(np.uint8)
# this mask is again made specifically for the stock footage

## Final Code

In [8]:
cap = cv2.VideoCapture("people.mp4")  # For Video
 
totalCountUp = []
totalCountDown = []
 
while True:
    success, img = cap.read()
    if not success:
        break  # Break the loop if there are no more frames
    
    # Apply the mask to the frame
    masked_frame = cv2.bitwise_and(img, img, mask=mask)

    # Process results
 
    imgGraphics = cv2.imread("graphics.png", cv2.IMREAD_UNCHANGED)
    img = cvzone.overlayPNG(img, imgGraphics, (730, 260))
    results = model(masked_frame, stream=True)
 
    detections = np.empty((0, 5))
 
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Bounding Box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            # cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,255),3)
            w, h = x2 - x1, y2 - y1
 
            # Confidence
            conf = math.ceil((box.conf[0] * 100)) / 100
            # Class Name
            cls = int(box.cls[0])
            # currentClass = coco_classes[cls]
 
            if coco_classes[cls] == "Person" and conf > 0.3:
                # cvzone.putTextRect(img, f'{currentClass} {conf}', (max(0, x1), max(35, y1)),
                #                    scale=0.6, thickness=1, offset=3)
                # cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=5)
                currentArray = np.array([x1, y1, x2, y2, conf])
                detections = np.vstack((detections, currentArray))
 
    resultsTracker = tracker.update(detections)
 
    cv2.line(img, (limitsUp[0], limitsUp[1]), (limitsUp[2], limitsUp[3]), (0, 0, 255), 5)
    cv2.line(img, (limitsDown[0], limitsDown[1]), (limitsDown[2], limitsDown[3]), (0, 0, 255), 5)
    for result in resultsTracker:
        x1, y1, x2, y2, id = result
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        print(result)
        w, h = x2 - x1, y2 - y1
        cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=2, colorR=(255, 0, 255))
        cvzone.putTextRect(img, f' {int(id)}', (max(0, x1), max(35, y1)),
                           scale=2, thickness=3, offset=10)
 
        cx, cy = x1 + w // 2, y1 + h // 2
        cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)

        # for people going up
        if limitsUp[0] < cx < limitsUp[2] and limitsUp[1] - 15 < cy < limitsUp[1] + 15:
            if totalCountUp.count(id) == 0:
                totalCountUp.append(id)
                cv2.line(img, (limitsUp[0], limitsUp[1]), (limitsUp[2], limitsUp[3]), (0, 255, 0), 5)
         # for people going down
        if limitsDown[0] < cx < limitsDown[2] and limitsDown[1] - 15 < cy < limitsDown[1] + 15:
            if totalCountDown.count(id) == 0:
                totalCountDown.append(id)
                cv2.line(img, (limitsDown[0], limitsDown[1]), (limitsDown[2], limitsDown[3]), (0, 255, 0), 5)
         
    # cvzone.putTextRect(img, f' Count: {len(totalCount)}', (50, 50))
    cv2.putText(img,str(len(totalCountUp)),(929,345),cv2.FONT_HERSHEY_PLAIN,5,(139,195,75),7)
    cv2.putText(img,str(len(totalCountDown)),(1191,345),cv2.FONT_HERSHEY_PLAIN,5,(50,50,230),7)
 
    cv2.imshow("Image", img)
    # cv2.imshow("ImageRegion", imgRegion)

 # Wait for key press (display frame for 1 millisecond)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit , waitKey os kept 0 , so we can move
        #frame by frame
        break

# Release resources
cap.release()
cv2.destroyAllWindows()





0: 384x640 2 persons, 71.1ms
Speed: 5.0ms preprocess, 71.1ms inference, 193.6ms postprocess per image at shape (1, 3, 384, 640)
[        402         646         525         719           2]
[        234         320         353         580           1]

0: 384x640 2 persons, 51.1ms
Speed: 3.0ms preprocess, 51.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
[        402         646         525         719           2]
[        234         320         353         580           1]

0: 384x640 2 persons, 50.8ms
Speed: 1.9ms preprocess, 50.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
[     401.14       643.4      527.74      718.78           2]
[     232.23      317.09      351.95       578.2           1]

0: 384x640 2 persons, 51.8ms
Speed: 2.0ms preprocess, 51.8ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
[     401.14         640      530.61      718.29           2]
[     230.33      313.78      350.81       575.6        

## THE END